In [1]:
import os
import hail as hl
import pyspark
import bokeh
import logging
import random
import pandas as pd
import numpy as np
from scipy import stats
import pickle 
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union
from hail.plot import show, output_notebook
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
plot_dir = "/home/ubuntu/data/tmp"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference='GRCh38')
output_notebook()
logging.basicConfig(format="%(levelname)s (%(name)s %(lineno)s): %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.5
SparkUI available at http://spark-master:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.41-b8144dba46e6
LOGGING: writing to /home/ubuntu/data/tmp/scripts/sanger_gnomad_hail_qc/notebooks/hail-20201027-1758-0.2.41-b8144dba46e6.log


Loading BokehJS ...

In [6]:
bed_to_exclude_pca = hl.import_bed(
        'file:///home/ubuntu/data/tmp/1000g/price_high_ld.bed.txt', reference_genome='GRCh38')

2020-10-27 18:00:21 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (user-specified)
  Loading column 'f1' as type 'int32' (user-specified)
  Loading column 'f2' as type 'int32' (user-specified)



In [7]:
bed2=hl.import_locus_intervals('file:///home/ubuntu/data/tmp/1000g/price_high_ld.bed.txt', reference_genome="GRCh38")

2020-10-27 18:01:42 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (user-specified)
  Loading column 'f1' as type 'int32' (user-specified)
  Loading column 'f2' as type 'int32' (user-specified)



In [9]:
bed_to_exclude_pca.show()

interval
interval<locus<GRCh38>>
[chr1:47534329-chr1:51534329)
[chr2:85772878-chr2:99883539)
[chr2:133742430-chr2:137242431)
[chr2:182135274-chr2:189135275)
[chr3:47458511-chr3:49962568)
[chr3:83450850-chr3:86950851)
[chr3:88950851-chr3:97781157)
[chr5:44499899-chr5:51204167)
[chr5:98664297-chr5:101164297)


In [10]:
bed2.show()

interval
interval<locus<GRCh38>>
[chr1:47534328-chr1:51534328]
[chr2:85772877-chr2:99883538]
[chr2:133742429-chr2:137242430]
[chr2:182135273-chr2:189135274]
[chr3:47458510-chr3:49962567]
[chr3:83450849-chr3:86950850]
[chr3:88950850-chr3:97781156]
[chr5:44499898-chr5:51204166]
[chr5:98664296-chr5:101164296]
